In [ ]:

"""
Hospital Patient Manager
• Add patient  (enhanced)
• View all patients
• Search by patient ID
• Save & exit
-------------------------------------------------
File: patients.csv  (created automatically)
"""

import csv, os, datetime
from tabulate import tabulate          # pip install tabulate   (once)

CSV_FILE = "patients.csv"


# ────────────────────────────────
# Persistence helpers
# ────────────────────────────────
def load_data():
    if not os.path.exists(CSV_FILE):
        return []
    with open(CSV_FILE, newline="") as f:
        reader = csv.DictReader(f)
        return list(reader)


def save_data(patients):
    """Write out to CSV (overwrites)."""
    if not patients:
        # create empty header so the file still exists
        with open(CSV_FILE, "w", newline="") as f:
            writer = csv.DictWriter(
                f,
                fieldnames=["ID", "Name", "Age", "Condition", "AdmissionDate"],
            )
            writer.writeheader()
        return

    with open(CSV_FILE, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=patients[0].keys())
        writer.writeheader()
        writer.writerows(patients)
    print("💾  Data saved.")


def generate_id(patients):
    """Auto-increment ID like P001, P002 …"""
    next_num = len(patients) + 1
    return "P" + str(next_num).zfill(3)


# ────────────────────────────────
# Core features
# ────────────────────────────────
def add_patient(patients):
    print("\n--- Patient Registration  (type X to cancel at any prompt) ---")

    # 1  Name
    name = input("Full name: ").strip()
    if name.upper() == "X":
        print("⛔  Registration cancelled.\n")
        return
    if not name:
        print("❌  Name cannot be empty.\n")
        return

    # 2  Age  (0–130)
    age = input("Age (0-130): ").strip()
    if age.upper() == "X":
        print("⛔  Registration cancelled.\n")
        return
    if not (age.isdigit() and 0 <= int(age) <= 130):
        print("❌  Invalid age – enter a number from 0 to 130.\n")
        return

    # 3  Condition
    condition = input("Condition: ").strip()
    if condition.upper() == "X":
        print("⛔  Registration cancelled.\n")
        return
    if not condition:
        print("❌  Condition cannot be empty.\n")
        return

    # 4  Admission date  (defaults to today)
    date_str = input("Admission date (YYYY-MM-DD, blank = today): ").strip()
    if date_str.upper() == "X":
        print("⛔  Registration cancelled.\n")
        return
    if not date_str:
        date_str = datetime.date.today().isoformat()
    else:
        try:
            datetime.datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            print("❌  Date must be in YYYY-MM-DD format.\n")
            return

    # 5  Recap & confirmation
    new_id = generate_id(patients)
    print("\nPlease review:")
    print(tabulate(
        [[new_id, name, age, condition, date_str]],
        headers=["ID", "Name", "Age", "Condition", "AdmissionDate"],
        tablefmt="github",
    ))
    if input("Save this record? (Y/N): ").strip().upper() != "Y":
        print("⛔  Registration cancelled.\n")
        return

    # 6  Commit
    patients.append(
        {
            "ID": new_id,
            "Name": name,
            "Age": age,
            "Condition": condition,
            "AdmissionDate": date_str,
        }
    )
    save_data(patients)
    print("✅  Patient saved.\n")


def view_all_patients(patients):
    if not patients:
        print("⚠  No patients in the system yet.\n")
        return
    print("\n=== All Patients ===")
    print(tabulate(patients, headers="keys", tablefmt="github"))
    print()


def find_patient_by_id(patients, pid):
    for p in patients:
        if p["ID"].upper() == pid.upper():
            return p
    return None


def search_patient(patients):
    pid = input("Enter patient ID (e.g. P001): ").strip()
    if not pid:
        print("❌  ID cannot be empty.\n")
        return
    record = find_patient_by_id(patients, pid)
    if record:
        print(tabulate([record], headers="keys", tablefmt="github"))
        print()
    else:
        print("❌  No patient with ID " + pid + ".\n")

        # Edit patient information
def edit_patient(patients):
    patient_id = input("Enter patient ID to edit: ")
    if patient_id in patients:
        print("Enter new data (press Enter to keep current value):")
        name = input(f"Name ({patients[patient_id]['name']}): ") or patients[patient_id]['name']
        age = input(f"Age ({patients[patient_id]['age']}): ") or patients[patient_id]['age']
        condition = input(f"Condition ({patients[patient_id]['condition']}): ") or patients[patient_id]['condition']
        patients[patient_id] = {"name": name, "age": age, "condition": condition}
        print("Patient information updated.")
    else:
        print("Patient not found.")



# ────────────────────────────────
# Main loop
# ────────────────────────────────
# ────────────────────────────────
# Edit existing patient
# ────────────────────────────────
def edit_patient(patients):
    """Locate a patient by ID then let the user modify one or more fields."""
    pid = input("Enter Patient ID to edit: ").strip()
    record = next((p for p in patients if p["ID"] == pid), None)

    if not record:
        print("❌  No patient with ID " + pid + ".\n")
        return

    print("\nCurrent data:")
    print(tabulate([record], headers="keys", tablefmt="github"))
    print("Press Enter to keep a value unchanged; type X to cancel.\n")

    # name
    new_name = input("New name [" + record["Name"] + "]: ").strip()
    if new_name.upper() == "X":
        print("✖  Edit cancelled.\n")
        return
    if new_name:
        record["Name"] = new_name

    # age
    new_age = input("New age [" + record['Age'] + "]: ").strip()
    if new_age.upper() == "X":
        print("✖  Edit cancelled.\n")
        return
    if new_age:
        if new_age.isdigit() and 0 <= int(new_age) <= 130:
            record["Age"] = new_age
        else:
            print("❌  Invalid age – edit aborted.\n")
            return

    # condition
    new_cond = input("New condition [" + record["Condition"] + "]: ").strip()
    if new_cond.upper() == "X":
        print("✖  Edit cancelled.\n")
        return
    if new_cond:
        record["Condition"] = new_cond

    # admission date
    new_date = input(
        "New admission date [" + record["AdmissionDate"] + "] (YYYY-MM-DD): "
    ).strip()
    if new_date.upper() == "X":
        print("✖  Edit cancelled.\n")
        return
    if new_date:
        try:
            datetime.datetime.strptime(new_date, "%Y-%m-%d")
            record["AdmissionDate"] = new_date
        except ValueError:
            print("❌  Bad date – edit aborted.\n")
            return

    print("\n✔  Record updated:\n")
    print(tabulate([record], headers="keys", tablefmt="github"))
    print()
    
def main():
    patients = load_data()

    while True:
        print("1) Add patient")
        print("2) View all patients")
        print("3) Search by patient ID")
        print("4) Save & exit")
        print("5) Edit patient information")
        choice = input("Select option: ").strip()

        if choice == "1":
            add_patient(patients)
        elif choice == "2":
            view_all_patients(patients)
        elif choice == "3":
            search_patient(patients)
        elif choice == "4":
            save_data(patients)
            print("Good-bye!")
            break
        elif choice == "5":
            edit_patient(patients)
        else:
            print("Invalid selection.\n")


if __name__ == "__main__":
    main()

1) Add patient
2) View all patients
3) Search by patient ID
4) Save & exit
5) Edit patient information

--- Patient Registration  (type X to cancel at any prompt) ---

Please review:
| ID   | Name             |   Age | Condition    | AdmissionDate   |
|------|------------------|-------|--------------|-----------------|
| P003 | mohammedd ksanjs |    24 | dsjcdflvjmfd | 2012-12-12      |
⛔  Registration cancelled.

1) Add patient
2) View all patients
3) Search by patient ID
4) Save & exit
5) Edit patient information
❌  No patient with ID p003.

1) Add patient
2) View all patients
3) Search by patient ID
4) Save & exit
5) Edit patient information

=== All Patients ===
| ID   | Name          |   Age | Condition   | AdmissionDate   |
|------|---------------|-------|-------------|-----------------|
| P001 | mohammed rami |    27 | headache    | 2012-05-11      |
| P002 | ahmed omar    |    15 | diahreaa    | 2016-12-11      |

1) Add patient
2) View all patients
3) Search by patient ID
4) S